In [132]:
import sys

import numpy as np
import cvxpy as cp
import math

np.set_printoptions(precision=3)

In [120]:
# Takes 2-D agent value np.ndarray and 1-D outside option np.ndarray
# Returns Probability Distribution that satisfies Nash Bargaining Solution
def NBS(val_mat, O_mat, prob_consts=1.0):
    if type(val_mat) != np.ndarray:
        raise Exception("val_mat must be type np.ndarray. Current type: ", type(val_mat))
    if type(O_mat) != np.ndarray:
        raise Exception("O must be type np.ndarray. Current type: ", type(O))
    if np.ndim(val_mat) != 2:
        raise Exception("val_mat must be a 2-D array. Current Shape: ", val_mat.shape)
    if np.ndim(O_mat) != 1:
        raise Exception("O must be a 1-D array. Current Shape: ", O.shape)
        
    V = cp.Parameter(val_mat.shape, nonneg=True)
    V.value = val_mat
    
    O = cp.Parameter(O_mat.shape)
    O.value = O_mat
    
    # create matrix of same shape holding probability variables
    P = cp.Variable(nonneg=True, shape=V.shape)
    U = cp.Variable(nonneg=True, shape=(V.shape[0],))
    

    # Setting Objective Function
    objective_fn = cp.sum(-cp.log(U))
#     objective_fn = 0
#     for i in range(U.shape[0]):
#         objective_fn -= cp.log(U[i]-O[i]) #_______________________________
#     print("objective_fn: ", objective_fn)
    
    # Setting constraint that ∑_{j}p for all i <= 1
    constraints = []
    for i in range(V.shape[0]):
        constraints.append(cp.sum(cp.multiply(P[i], V[i]))-O[i]-U[i]==0.0)
        constraints.append(cp.sum(cp.multiply(P[i], V[i]))-O[i] >= 0.0)
        constraints.append(cp.sum(P[i])<=prob_consts)

    # Setting constraint that ∑_{i} p for all j <= 1
    for j in range(V.shape[1]):
        col_sum = 0
        for i in range(V.shape[0]):
            col_sum += P[i][j]
        constraints.append(col_sum <= prob_consts)

    problem = cp.Problem(cp.Minimize(objective_fn), constraints)
    problem.solve()
    
    if P.value is None:
        return P.value

    P_vals = np.array(P.value)

    return P_vals

In [96]:
A = np.array([[0.59024284, 0.27186421, 0.30129857, 0.68020923, 0.87239912],
 [0.19124566, 0.29134386, 0.65605424, 0.49163814, 0.94681575],
 [0.9039715,  0.20068686, 0.58048439, 0.77656511, 0.57591704]])
O = np.zeros(A.shape[0])

cap = np.ones(A.shape[1])
for i in range(O.size):
    O[i] = np.sum(np.multiply(A[i], cap)) / 5
print("O shape: ", O.shape)
print("A shape: ", A.shape)
print("Array: \n", A)
print("Agent Sum Vals: ", np.sum(A, axis=1))
print("O: \n", O)
for i in range(3):
    print("Agent ", i, " uniform utility: ", np.sum(A[i] * 0.2))
nb = NBS(A, O)
print("NBS: \n", nb)

O shape:  (3,)
A shape:  (3, 5)
Array: 
 [[0.59024284 0.27186421 0.30129857 0.68020923 0.87239912]
 [0.19124566 0.29134386 0.65605424 0.49163814 0.94681575]
 [0.9039715  0.20068686 0.58048439 0.77656511 0.57591704]]
Agent Sum Vals:  [2.71601397 2.57709765 3.0376249 ]
O: 
 [0.54320279 0.51541953 0.60752498]
Agent  0  uniform utility:  0.5432027939999999
Agent  1  uniform utility:  0.5154195300000001
Agent  2  uniform utility:  0.6075249800000001
NBS: 
 [[4.02770906e-09 0.00000000e+00 0.00000000e+00 6.14596475e-01
  3.85403520e-01]
 [0.00000000e+00 1.02554565e-09 3.85403516e-01 5.37162034e-09
  6.14596476e-01]
 [9.99999978e-01 0.00000000e+00 2.38866991e-09 1.86962148e-08
  0.00000000e+00]]


In [102]:
B = np.array([[0.78268411, 0.31431919, 0.01601237, 0.23318279, 0.49309724],
 [0.39540977, 0.29893196, 0.49814774, 0.85194549, 0.3870454 ],
 [0.47406927, 0.91671614, 0.397393,   0.2115808,  0.73576799],
             [0,0,0,0,0], [0,0,0,0,0]])

Ob = np.zeros(B.shape[0])

cap = np.ones(B.shape[1])
for i in range(Ob.size):
    Ob[i] = np.sum(np.multiply(B[i], cap)) / B.shape[1]
    
print("O shape: ", Ob.shape)
print("B shape: ", B.shape)
print("Array: \n", B)
print("Agent Sum Vals: ", np.sum(B, axis=1))
print("Ob: \n", Ob)
for i in range(3):
    print("Agent ", i, " uniform utility: ", np.sum(B[i] * 0.2))
nbb = NBS(B, Ob)
print("NBS: \n", nbb)

O shape:  (5,)
B shape:  (5, 5)
Array: 
 [[0.78268411 0.31431919 0.01601237 0.23318279 0.49309724]
 [0.39540977 0.29893196 0.49814774 0.85194549 0.3870454 ]
 [0.47406927 0.91671614 0.397393   0.2115808  0.73576799]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]
Agent Sum Vals:  [1.8392957  2.43148036 2.7355272  0.         0.        ]
Ob: 
 [0.36785914 0.48629607 0.54710544 0.         0.        ]
Agent  0  uniform utility:  0.36785914000000003
Agent  1  uniform utility:  0.486296072
Agent  2  uniform utility:  0.54710544


SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

In [106]:
def pref_att(num_agents, num_items, p):
    array = np.zeros(shape=(num_agents, num_items))
    array[0] = np.random.rand(num_items)
    for i in range(1, num_agents):
        rint = np.random.randint(0, i)
        array[i] = array[rint]
        for j in range(1, num_items):
            r = np.random.rand()
            if r < p:
                array[i][j] = np.random.rand()
    return array

In [142]:
ar = pref_att(8,8,1.0)
half_ar = ar[0:4]/2
print("selected: \n", half_ar)
caps = np.ones(8)
avail_caps = caps/2

O = np.zeros(4)
for i in range(4):
    O[i] = np.sum(np.multiply(avail_caps, half_ar[i]))/8
print("O: \n", O)


nbs = NBS(half_ar, O, 0.5)
print("NBS: \n", nbs)
print("Agent sums: \n", np.sum(nbs, axis=1))
print("Item sums: \n", np.sum(nbs, axis=0))

selected: 
 [[0.464 0.183 0.408 0.397 0.49  0.042 0.247 0.066]
 [0.464 0.416 0.212 0.001 0.482 0.066 0.039 0.318]
 [0.464 0.106 0.131 0.463 0.049 0.372 0.492 0.056]
 [0.464 0.013 0.267 0.44  0.442 0.207 0.491 0.453]]
O: 
 [0.144 0.125 0.133 0.174]
NBS: 
 [[1.951e-07 6.935e-10 3.345e-08 1.928e-08 5.000e-01 0.000e+00 1.076e-09
  0.000e+00]
 [5.000e-01 7.602e-08 1.711e-09 0.000e+00 2.213e-07 0.000e+00 0.000e+00
  6.732e-09]
 [4.057e-08 0.000e+00 0.000e+00 5.000e-01 0.000e+00 9.060e-09 1.467e-07
  0.000e+00]
 [3.681e-08 0.000e+00 4.560e-10 2.753e-08 1.015e-08 0.000e+00 5.000e-01
  9.788e-08]]
Agent sums: 
 [0.5 0.5 0.5 0.5]
Item sums: 
 [5.000e-01 7.671e-08 3.562e-08 5.000e-01 5.000e-01 9.060e-09 5.000e-01
 1.046e-07]


In [134]:
type(sys.float_info.epsilon)

float